# Support Vector Machine Classifier

## The overall video ratings for `Observer`(V1), `Strategy`(V2) and `MVC`(V3) patterns will be classified separately based on each learning style dimension scores below...
* A/R_Score
* S/I_Score
* Vi/Vb_Score
* S/G_Score

The classifiers were trained using `GridSearchCV`, a hyperparamater tuning technique, and the training set was split into `10-fold cross validation`. The average scores were collected after each classifier was 'train test split' a few times (various random states) with train-test.

# Importing the necessary libraries

In [163]:
### For data manipulation and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## To encode and scale the data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

### Machine Learning
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE,RandomOverSampler

# Reading the dataset

In [164]:
pd.set_option('display.max_columns', None)
training = pd.read_csv('BDP_Cleaned.csv') #dataset after feature selection process for 5-Level Classification

# Dropping all irrelevant features
* `Time_Day_Watched` of all design patterns as features have already been extracted
* `Gender` as male and females are imbalanced and does not provide any useful insights
* `Duration` of each video as watch status and percentage duration have already been extracted from it

In [165]:
training.drop(['Time_Day_Watched_(V1-1)',
         'Time_Day_Watched_(V1-2)',
         'Time_Day_Watched_(V1-3)',
         'Time_Day_Watched_(V1-4)',
         'Time_Day_Watched_(V2-1)',
         'Time_Day_Watched_(V2-2)',
         'Time_Day_Watched_(V2-3)',
         'Time_Day_Watched_(V2-4)',
         'Time_Day_Watched_(V3-1)',
         'Time_Day_Watched_(V3-2)',
         'Time_Day_Watched_(V3-3)',
         'Time_Day_Watched_(V3-4)',
               'Gender',
               "V1-1_(1)",
               "V1-2_(1)",
               "V1-3_(1)",
               "V1-4_(1)",
               "V2-1_(2)",
               "V2-1_(2)",
               "V2-2_(2)",
               "V2-3_(2)",
               "V2-4_(2)",
               "V3-1_(3)",
               "V3-2_(3)",
               "V3-3_(3)",
               "V3-4_(3)",
               
        ], axis = 1,inplace = True)

# Getting numerical and categorical features for further tuning scaling and fitting

In [166]:
numerical_feats = training.dtypes[training.dtypes != "object"].index
categorical_feats = training.dtypes[training.dtypes == "object"].index

In [167]:
print("*"*100)
print(training[numerical_feats].columns)
print("*"*100)
print(training[categorical_feats].columns)
print("*"*100)
training.columns

****************************************************************************************************
Index(['V1_PercentWatched', 'V2_PercentWatched', 'V3_PercentWatched',
       'A/R_Score', 'Active', 'Reflective', 'S/I_Score', 'Sensing',
       'Intuitive', 'Vi/Vb_Score', 'Visual', 'Verbal', 'S/G_Score',
       'Sequential', 'Global', 'Quiz'],
      dtype='object')
****************************************************************************************************
Index(['CGPA_Class', 'Student_Rating_(V1-1)', 'Student_Rating_(V1-2)',
       'Student_Rating_(V1-3)', 'Student_Rating_(V1-4)', 'Overall_Rating_V1',
       'Watch_Status_V1', 'Student_Rating_(V2-1)', 'Student_Rating_(V2-2)',
       'Student_Rating_(V2-3)', 'Student_Rating_(V2-4)', 'Overall_Rating_V2',
       'Watch_Status_V2', 'Student_Rating_(V3-1)', 'Student_Rating_(V3-2)',
       'Student_Rating_(V3-3)', 'Student_Rating_(V3-4)', 'Overall_Rating_V3',
       'Watch_Status_V3', 'part_day_V1-1', 'part_day_V1-2', 'part_day_V1-

Index(['CGPA_Class', 'Student_Rating_(V1-1)', 'Student_Rating_(V1-2)',
       'Student_Rating_(V1-3)', 'Student_Rating_(V1-4)', 'Overall_Rating_V1',
       'Watch_Status_V1', 'V1_PercentWatched', 'Student_Rating_(V2-1)',
       'Student_Rating_(V2-2)', 'Student_Rating_(V2-3)',
       'Student_Rating_(V2-4)', 'Overall_Rating_V2', 'Watch_Status_V2',
       'V2_PercentWatched', 'Student_Rating_(V3-1)', 'Student_Rating_(V3-2)',
       'Student_Rating_(V3-3)', 'Student_Rating_(V3-4)', 'Overall_Rating_V3',
       'Watch_Status_V3', 'V3_PercentWatched', 'A/R_Score', 'Active',
       'Reflective', 'S/I_Score', 'Sensing', 'Intuitive', 'Vi/Vb_Score',
       'Visual', 'Verbal', 'S/G_Score', 'Sequential', 'Global', 'Quiz',
       'part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4',
       'part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4',
       'part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4'],
      dtype='object')

# Using MinMax Scaler to scale data from 0 to 1 since numerical features are not normally distributed

In [168]:
scaler = MinMaxScaler()
training['Quiz'] = scaler.fit_transform(training[['Quiz']])
training['A/R_Score'] = scaler.fit_transform(training[['A/R_Score']])
training['S/I_Score'] = scaler.fit_transform(training[['S/I_Score']])
training['Vi/Vb_Score'] = scaler.fit_transform(training[['Vi/Vb_Score']])
training['S/G_Score'] = scaler.fit_transform(training[['S/G_Score']])
training['V1_PercentWatched'] = scaler.fit_transform(training[['V1_PercentWatched']])
training['V2_PercentWatched'] = scaler.fit_transform(training[['V2_PercentWatched']])
training['V3_PercentWatched'] = scaler.fit_transform(training[['V3_PercentWatched']])
training.head()

,CGPA_Class,Student_Rating_(V1-1),Student_Rating_(V1-2),Student_Rating_(V1-3),Student_Rating_(V1-4),Overall_Rating_V1,Watch_Status_V1,V1_PercentWatched,Student_Rating_(V2-1),Student_Rating_(V2-2),Student_Rating_(V2-3),Student_Rating_(V2-4),Overall_Rating_V2,Watch_Status_V2,V2_PercentWatched,Student_Rating_(V3-1),Student_Rating_(V3-2),Student_Rating_(V3-3),Student_Rating_(V3-4),Overall_Rating_V3,Watch_Status_V3,V3_PercentWatched,A/R_Score,Active,Reflective,S/I_Score,Sensing,Intuitive,Vi/Vb_Score,Visual,Verbal,S/G_Score,Sequential,Global,Quiz,part_day_V1-1,part_day_V1-2,part_day_V1-3,part_day_V1-4,part_day_V2-1,part_day_V2-2,part_day_V2-3,part_day_V2-4,part_day_V3-1,part_day_V3-2,part_day_V3-3,part_day_V3-4
0,2.67 - 3.32,Excellent,Satisfactory,Very Good,Very Good,Very Good,Completed,0.747748,Very Good,Satisfactory,Very Good,Very Good,Very Good,Completed,0.440789,Excellent,Very Good,Very Good,Very Good,Very Good,Completed,0.632653,0.2,1,0,0.25,1,0,0.6,1,0,0.0,1,0,1.000,night,night,night,night,night,night,night,night,night,night,night,night
1,2.67 - 3.32,Very Good,Very Good,Very Good,Very Good,Very Good,Completed,0.747748,Very Good,Very Good,Very Good,Very Good,Very Good,Completed,0.440789,Very Good,Very Good,Very Good,Very Good,Very Good,Rewatched,0.653061,0.2,0,1,1.00,1,0,0.6,1,0,0.2,0,1,0.750,midnight,night,night,night,night,night,night,night,night,night,night,night
2,2.67 - 3.32,Excellent,Satisfactory,Very Good,Satisfactory,Very Good,Completed,0.747748,Very Good,Satisfactory,Very Good,Excellent,Very Good,Completed,0.440789,Excellent,Very Good,Very Good,Very Good,Very Good,Completed,0.632653,0.0,0,1,0.75,1,0,0.6,1,0,0.2,0,1,0.500,night,night,afternoon,afternoon,night,night,afternoon,afternoon,night,night,afternoon,afternoon
3,3.67 - 4.00,Excellent,Very Good,Excellent,Excellent,Excellent,Completed,0.747748,Excellent,Excellent,Very Good,Excellent,Excellent,Rewatched,0.618421,Excellent,Excellent,Very Good,Excellent,Excellent,Completed,0.632653,0.2,0,1,0.25,1,0,0.6,1,0,0.0,1,0,0.375,midnight,midnight,midnight,afternoon,midnight,midnight,midnight,midnight,midnight,midnight,midnight,midnight
4,2.67 - 3.32,Very Good,Very Good,Satisfactory,Very Good,Very Good,Completed,0.747748,Excellent,Very Good,Very Good,Satisfactory,Very Good,Completed,0.440789,Very Good,Very Good,Excellent,Very Good,Very Good,Completed,0.632653,0.2,1,0,0.00,1,0,0.4,1,0,0.2,0,1,0.750,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon


# Encoding nominal values using label encoder

In [169]:
features =  ["CGPA_Class",
             "Overall_Rating_V1",
             "Overall_Rating_V2",
             "Overall_Rating_V3",
             "Watch_Status_V1",
             "Watch_Status_V2",
             "Watch_Status_V3",
             "part_day_V1-1",
             "part_day_V1-2",
             "part_day_V1-3",
             "part_day_V1-4",
             "part_day_V2-1",
             "part_day_V2-2",
             "part_day_V2-3",
             "part_day_V2-4",
             "part_day_V3-1",
             "part_day_V3-2",
             "part_day_V3-3",
             "part_day_V3-4"]

for i in features:
    training.loc[:,i] = LabelEncoder().fit_transform(training.loc[:,i]) 

### Overall_rating_V1 `(Observer)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [170]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'A/R_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_A = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_A

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.806122,0.811799,0.806122,0.807595,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.785714,0.791156,0.785714,0.786969,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.683673,0.718195,0.683673,0.677871,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.797297,0.810428,0.797297,0.799417,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.770270,0.794042,0.770270,0.772556,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.770270,0.768769,0.770270,0.769266,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.755102,0.770408,0.755102,0.757293,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.795918,0.808078,0.795918,0.795077,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.836735,0.838678,0.836735,0.834647,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"


In [171]:
df_A.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.795918,0.805721,0.795918,0.795673
0.3,0.779279,0.791079,0.779279,0.780413
0.4,0.758503,0.773717,0.758503,0.757478


### Overall_rating_V1 `(Observer)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)MVC

In [172]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'S/I_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_B = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_B

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.806122,0.823178,0.806122,0.809197,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.755102,0.745294,0.755102,0.737897,"{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}"
2,0.4,101,0.714286,0.766600,0.714286,0.704425,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.810811,0.822717,0.810811,0.814218,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.770270,0.771918,0.770270,0.765870,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.783784,0.797297,0.783784,0.785471,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.816327,0.825656,0.816327,0.810745,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.857143,0.851992,0.857143,0.852195,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.836735,0.836735,0.836735,0.836735,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [173]:
df_B.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.836735,0.838128,0.836735,0.833225
0.3,0.788288,0.797311,0.788288,0.788520
0.4,0.758503,0.778357,0.758503,0.750506


### Overall_rating_V1 `(Observer)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [174]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'Vi/Vb_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_C = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_C

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.785714,0.799320,0.785714,0.780794,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.755102,0.754866,0.755102,0.736978,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.724490,0.803829,0.724490,0.729795,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.756757,0.755903,0.756757,0.746095,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.729730,0.718507,0.729730,0.701122,"{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}"
5,0.3,101,0.770270,0.797551,0.770270,0.746323,"{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}"
6,0.2,7,0.693878,0.671720,0.693878,0.669546,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.836735,0.840136,0.836735,0.834614,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.816327,0.830717,0.816327,0.809595,"{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}"


In [175]:
df_C.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.782313,0.780858,0.782313,0.771252
0.3,0.752252,0.757320,0.752252,0.731180
0.4,0.755102,0.786005,0.755102,0.749189


### Overall_rating_V1 `(Observer)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [176]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'S/G_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_D = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_D

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.765306,0.789716,0.765306,0.754926,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.775510,0.783838,0.775510,0.777277,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.775510,0.808001,0.775510,0.768768,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.783784,0.787838,0.783784,0.785527,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.743243,0.759951,0.743243,0.746576,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.810811,0.854118,0.810811,0.800313,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.755102,0.771078,0.755102,0.759343,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.795918,0.811224,0.795918,0.800454,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.897959,0.902385,0.897959,0.893574,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"


In [177]:
df_D.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.816327,0.828229,0.816327,0.817790
0.3,0.779279,0.800636,0.779279,0.777472
0.4,0.772109,0.793852,0.772109,0.766990


### Overall_rating_V2 `(Strategy)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [178]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'A/R_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_E = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_E

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.760000,0.791791,0.760000,0.762197,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.780000,0.785744,0.780000,0.780720,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.810000,0.804167,0.810000,0.805655,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.720000,0.737486,0.720000,0.722889,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.800000,0.801577,0.800000,0.798546,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.733333,0.741520,0.733333,0.730484,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.700000,0.703820,0.700000,0.698668,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.840000,0.879111,0.840000,0.849091,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.860000,0.864545,0.860000,0.859333,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [179]:
df_E.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.800000,0.815825,0.800000,0.802364
0.3,0.751111,0.760194,0.751111,0.750640
0.4,0.783333,0.793901,0.783333,0.782857


### Overall_rating_V2 `(Strategy)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [180]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'S/I_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_F = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_F

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.760000,0.790108,0.760000,0.759793,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.730000,0.741939,0.730000,0.727229,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.790000,0.803246,0.790000,0.790579,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.786667,0.792349,0.786667,0.783717,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.720000,0.744033,0.720000,0.716725,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.920000,0.928698,0.920000,0.919596,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.820000,0.818571,0.820000,0.816923,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.780000,0.815431,0.780000,0.787178,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.880000,0.880909,0.880000,0.877333,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"


In [181]:
df_F.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.826667,0.838304,0.826667,0.827145
0.3,0.808889,0.821693,0.808889,0.806679
0.4,0.760000,0.778431,0.760000,0.759200


### Overall_rating_V2 `(Strategy)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [182]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'Vi/Vb_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_G = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_G

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.760000,0.790489,0.760000,0.765722,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.760000,0.765556,0.760000,0.759666,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.820000,0.835692,0.820000,0.824207,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.773333,0.778895,0.773333,0.773401,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.773333,0.801569,0.773333,0.783032,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.893333,0.897018,0.893333,0.894163,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.800000,0.792000,0.800000,0.789333,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.840000,0.861222,0.840000,0.845495,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.880000,0.902198,0.880000,0.881559,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [183]:
df_G.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.840000,0.851807,0.840000,0.838796
0.3,0.813333,0.825827,0.813333,0.816865
0.4,0.780000,0.797246,0.780000,0.783199


### Overall_rating_V2 `(Strategy)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [184]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'S/G_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_H = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_H

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.750000,0.787397,0.750000,0.739373,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.800000,0.811222,0.800000,0.803918,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.810000,0.806957,0.810000,0.807851,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.746667,0.761008,0.746667,0.749847,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.786667,0.818564,0.786667,0.797028,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.893333,0.896296,0.893333,0.894400,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.780000,0.773077,0.780000,0.775906,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.840000,0.931429,0.840000,0.852462,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.840000,0.866000,0.840000,0.825714,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"


In [185]:
df_H.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.820000,0.856835,0.820000,0.818027
0.3,0.808889,0.825290,0.808889,0.813758
0.4,0.786667,0.801858,0.786667,0.783714


### Overall_rating_V3 `(MVC)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [186]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'A/R_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_I = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_I

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.841667,0.837540,0.841667,0.836408,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.866667,0.869923,0.866667,0.859143,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.783333,0.787075,0.783333,0.781026,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.877778,0.870909,0.877778,0.871755,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.888889,0.892146,0.888889,0.883988,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.833333,0.823350,0.833333,0.824685,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.850000,0.836527,0.850000,0.840406,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.816667,0.815534,0.816667,0.813581,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.850000,0.859363,0.850000,0.848429,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [187]:
df_I.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.838889,0.837141,0.838889,0.834139
0.3,0.866667,0.862135,0.866667,0.860143
0.4,0.830556,0.831512,0.830556,0.825526


### Overall_rating_V3 `(MVC)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [188]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'S/I_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_J = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_J

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.825000,0.828850,0.825000,0.825941,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.841667,0.836371,0.841667,0.835831,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.808333,0.810857,0.808333,0.809018,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.833333,0.857018,0.833333,0.841270,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.844444,0.838044,0.844444,0.837636,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.855556,0.849563,0.855556,0.851804,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.900000,0.902778,0.900000,0.900794,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.883333,0.884470,0.883333,0.879812,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.866667,0.865726,0.866667,0.865656,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [189]:
df_J.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.883333,0.884325,0.883333,0.882087
0.3,0.844444,0.848208,0.844444,0.843570
0.4,0.825000,0.825359,0.825000,0.823597


### Overall_rating_V3 `(MVC)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [190]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'Vi/Vb_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_K = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_K

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.850000,0.846554,0.850000,0.847793,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.850000,0.845241,0.850000,0.842490,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.766667,0.778552,0.766667,0.768335,"{'C': 1, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.877778,0.874674,0.877778,0.875965,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.877778,0.874799,0.877778,0.871651,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.877778,0.874658,0.877778,0.873550,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.833333,0.832986,0.833333,0.832653,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.883333,0.894420,0.883333,0.877319,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.900000,0.896282,0.900000,0.896094,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [191]:
df_K.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.872222,0.874563,0.872222,0.868688
0.3,0.877778,0.874710,0.877778,0.873722
0.4,0.822222,0.823449,0.822222,0.819539


### Overall_rating_V3 `(MVC)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [192]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'S/G_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'C': [0.1, 1, 10, 100],
                      'gamma': [1, 0.1, 0.01, 0.001],
                      'kernel': ['rbf','sigmoid']}

        model = GridSearchCV(SVC(random_state=0), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_L = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_L

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits
Fitting 10 folds for each of 32 candidates, totalling 320 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.4,7,0.800000,0.802312,0.800000,0.798335,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
1,0.4,69,0.825000,0.821054,0.825000,0.816100,"{'C': 10, 'gamma': 1, 'kernel': 'rbf'}"
2,0.4,101,0.825000,0.818122,0.825000,0.817663,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
3,0.3,7,0.822222,0.818012,0.822222,0.818705,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
4,0.3,69,0.888889,0.893551,0.888889,0.884444,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
5,0.3,101,0.855556,0.854380,0.855556,0.849606,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
6,0.2,7,0.816667,0.807698,0.816667,0.809777,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
7,0.2,69,0.866667,0.871806,0.866667,0.861003,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"
8,0.2,101,0.883333,0.881984,0.883333,0.879569,"{'C': 100, 'gamma': 1, 'kernel': 'rbf'}"


In [193]:
df_L.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.855556,0.853829,0.855556,0.850117
0.3,0.855556,0.855314,0.855556,0.850919
0.4,0.816667,0.813830,0.816667,0.810699
